In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage
from mesostat.stat.anova import as_pandas, as_pandas_lst, anova_homebrew

from lib.sych.data_fc_db_raw import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.common.metric_helper import metric_by_session

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
# params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'

params['root_path_data'] = '/home/alyosha/data/sych_preprocessed/'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('sych_result_individual_region.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

# 1. Noise
## 1.1 SNR via AutoCorrelation

**Plan:**
* AutoCorr[1] is a number that is close to 1 if data is changing slowly, and close to zero if data changes very fast.

**FIXME**:
* Remove Performance above 1
* If performance becomes important, discuss with Y whether to use original or pooled performance, as they differ significantly

In [ ]:
metricName = 'autocorr_d1'
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    print(mousename)
    metric_by_session(dataDB, mc, ds, mousename, metricName, 'p', datatype='raw')

In [ ]:
imshow_dataset_by_mouse(dataDB, ds, 'autocorr_d1', limits=[0,1])

In [ ]:
ds.list_dsets_pd()

In [ ]:
from lib.analysis.snr import plot_snr_violins_bymouse

In [ ]:
plot_snr_violins_bymouse(dataDB, ds)

# SNR by paramsweep

In [ ]:
from lib.common.param_sweep import DataParameterSweep, param_vals_to_suffix, pd_row_to_kwargs

def snr_param_sweep(dataDB, ds, mc, skipExisting=False, exclQueryLst=None, **kwargs): 
    dataName = 'snr_mouse'
    metric = 'autocorr_d1'

    dps = DataParameterSweep(dataDB, exclQueryLst, mousename='auto', **kwargs)
    for idx, row in dps.sweepDF.iterrows():
        print(list(row))

        mousename = row['mousename']
        queryDict = dict(row)
        del queryDict['mousename']
        attrsDict = {**{'mousename': mousename}, **queryDict}
        attrsDict['metric'] = metric
        attrsDict['target_dim'] = '(channels,)'
        attrsDict['name'] = dataName

        dsDataLabels = ds.ping_data(dataName, attrsDict)
        if not skipExisting and len(dsDataLabels) > 0:
            dsuffix = dataName + '_' + '_'.join(attrsDict.values())
            print('Skipping existing', dsuffix)
        else:
            dataRSPLst = dataDB.get_neuro_data({'mousename': mousename}, **queryDict)

            dataRSP = np.concatenate(dataRSPLst, axis=0)
            
            mc.set_data(dataRSP, 'rsp')
            rez = mc.metric3D(metric, 'p')

#             ds.delete_rows(dsDataLabels, verbose=False)
            ds.save_data(dataName, rez, attrsDict)

In [ ]:
argSweepDict = {
    'datatype' : ['raw'],
    'intervName' : 'auto',
    'trialType' : 'auto',
    'performance' : 'auto'
}

In [ ]:
snr_param_sweep(dataDB, ds, mc, skipExisting=False, **argSweepDict)

In [ ]:
from mesostat.utils.pandas_helper import pd_query
from mesostat.visualization.mpl_matrix import plot_df_2D_outer_product

In [ ]:
df = ds.list_dsets_pd()
dfThis = pd_query(df, {'name': 'snr_mouse', 'metric': 'autocorr_d1'})
dfThis = dfThis[dfThis['intervName'] != 'PRE']
dfThis = dfThis.drop(['datatype', 'metric', 'target_dim', 'name', 'datetime', 'shape'], axis=1)
dfNew = pd.DataFrame()
for idx, row in dfThis.iterrows():
    data = ds.get_data(row['dset'])
    rowNew = row.copy().drop(['dset'])
    
    for i in range(len(data)):
        rowNew['channel'] = i
        rowNew['value'] = data[i]
        dfNew = dfNew.append(rowNew)
        
dfNew['channel'] = dfNew['channel'].astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
plot_df_2D_outer_product(ax, dfNew, ['channel'], ['performance', 'mousename', 'intervName', 'trialType'],
                         'value', vmin=0, vmax=1,
                         orderDict={'intervName': ['TEX', 'REW'], 'performance': ['naive', 'expert']})
plt.savefig('ac1_outer_01range.svg')
plt.show()